In [ ]:
import csv
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
from numpy import nan
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import roc_auc_score
import xgboost as xgb
plt.style.use('classic')
%matplotlib inline
pd.set_option('display.max_rows', None)

In [ ]:
df_train1 = pd.read_csv('sample_data/X_trainData_1.csv')#files must be uploaded to the sample_data folder in google colab
df_train2 = pd.read_csv('sample_data/Y_trainData_1.csv')
df_test = pd.read_csv('sample_data/X_testData_1.csv')
df_test = df_test.loc[:, df_test.columns != 'ID']


In [ ]:

print(df_train1.shape)
print(df_train2.shape)
print(df_test.shape)

(3393, 222)
(3393, 11)
(1000, 222)


In [ ]:
res = pd.concat([df_train1, df_train2], axis=1)
res = res[res.INH != -1]
res2= res[res.INH==1]
y = res['INH']
X = res.drop(['INH'], axis = 1) 
X = X.drop(['OFLX'], axis = 1) 
X = X.drop(['RIF'], axis = 1) 
X = X.drop(['CIP'], axis = 1) 
X = X.drop(['PZA'], axis = 1) 
X = X.drop(['EMB'], axis = 1)
X = X.drop(['CAP'], axis = 1)
X = X.drop(['STR'], axis = 1)
X = X.drop(['AMK'], axis = 1)
X = X.drop(['MOXI'], axis = 1)  
X = X.drop(['KAN'], axis = 1)    
print(y.shape)
print(X.shape)
print(res2.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.2)

(3356,)
(3356, 222)
(1832, 233)


In [ ]:
from sklearn.model_selection import GridSearchCV 
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

# Use the random grid to search for best hyperparameters

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 50)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 4, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X, y)
best_params=rf_random.best_params_
model = RandomForestClassifier(n_estimators=1200, criterion='entropy', max_depth=110, max_features= 'sqrt', min_samples_leaf= 2, min_samples_split=5, bootstrap=True, random_state=42)
model = RandomForestClassifier(n_estimators=best_params["n_estimators"], criterion='entropy', max_depth=best_params["max_depth"], max_features= 'sqrt', min_samples_leaf= best_params["min_samples_leaf"], min_samples_split= best_params["min_samples_split"], bootstrap=True, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict_proba(df_test)
# df = pd.read_csv('sample_data/Y_testData_1_nolabels_INH.csv')
# ID = df['ID'].tolist()
# print(y_pred.shape)
# # print(type(y_pred))
# # print(len(y_pred))

In [ ]:
# SVC Code with grid search Cv for hyperparam tuning
# defining parameter range 
# param_grid = {'C': [0.1,1,2,10,4,20],  
#               'gamma': [0.1,0.01,0.05,0.001,0.005], 
#               'kernel': ['rbf']}  
# grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 4,scoring='roc_auc') 
# # # fitting the model for grid search 
# grid.fit(X_train, y_train) 
# print(grid.best_params_) 
# model = RandomForestClassifier(n_estimators=100, criterion='entropy', max_depth=None)
model = SVC(C=3, kernel='rbf', degree=3, gamma=0.03,probability=True)
model.fit(X, y)
y_pred = model.predict_proba(df_test)
df = pd.read_csv('sample_data/Y_testData_1_nolabels_INH.csv')
ID = df['ID'].tolist()
print(y_pred.shape)
print(type(y_pred))
print(len(y_pred))

(1000, 2)
<class 'numpy.ndarray'>
1000


In [ ]:
cv_params = {'max_depth': [3,4,5,6,10], 'min_child_weight': [1,2,3,4], 'subsample': [0.8,0.9,1], 'max_delta_step': [0,1,2,4], 'learning_rate':  [0.1, 0.15, 0.2, 0.25, 0.3]}  
gscv = GridSearchCV(xgb.XGBClassifier(), cv_params, scoring = 'roc_auc', cv = 5)#running grid search cv on model 
gscv.fit(X_train, y_train)#fitting model
# gscv.grid_scores_#displaying scores
best_params= gscv.best_params_
print(best_params)
model = xgb.XGBClassifier(booster='dart',learning_rate=best_params["learning_rate"],max_depth=best_params["max_depth"],min_child_weight =best_params["min_child_weight"], eval_metric='auc',scale_pos_weight=1.2,max_delta_step=best_params["max_delta_step"],subsample=best_params["subsample"])
# model = xgb.XGBClassifier(learning_rate=0.26, min_child_weight=1, max_delta_step=2, max_depth=6, eval_metric='auc', subsample=1)
model.fit(X_train, y_train)

y_pred = model.predict_proba(df_test)#for converting into probabilities
df = pd.read_csv('sample_data/Y_testData_1_nolabels_INH.csv')
ID = df['ID'].tolist()
print(y_pred.shape)

(1000, 2)


In [ ]:
li_id = []
li_val = []
for i in ID:
    # print(i, y_pred[i-1])
    li_id.append(i)
    li_val.append((y_pred[i-1][1]))
csv = pd.DataFrame({'ID': li_id, 'INH' :li_val})
csv.to_csv('INH-submit.csv', index=False, header=True)    
y_pred = model.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))  
print(roc_auc_score(y_test, y_pred))

[[290  18]
 [  9 355]]
              precision    recall  f1-score   support

           0       0.97      0.94      0.96       308
           1       0.95      0.98      0.96       364

    accuracy                           0.96       672
   macro avg       0.96      0.96      0.96       672
weighted avg       0.96      0.96      0.96       672

0.9584165834165835
